In [1]:
import sys
sys.path.append("../")
from utils import *

In [2]:
os.listdir(data_path)

['tracks.csv',
 'albums.csv',
 '.DS_Store',
 'artists.csv',
 'pl_tracks.csv',
 'train_test_data.pkl']

In [3]:
df = pd.read_csv(data_path+'pl_tracks.csv')
print(df.shape)
df.head()

(66346428, 2)


,pid,track_uri
0,0_0,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,0_0,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,0_0,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,0_0,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,0_0,spotify:track:1lzr43nnXAijIGYnCT8M8H


In [4]:
# Filter the playlists by number of songs, >= 20, <= 100
n_songs = df.groupby('pid').count().reset_index()
n_songs.columns = ['pid', 'n_songs']
print("Number of pid before filtering:", len(n_songs.pid))
n_songs = n_songs[(n_songs['n_songs'] >= 20) & (n_songs['n_songs'] <= 100)]
pid = n_songs.pid.values.tolist()
pid = np.random.choice(pid, size = int(1000000 * 0.01), replace = False)
df = df[df['pid'].isin(pid)].reset_index(drop = True)
print("Number of pid after filtering:", len(pid))

Number of pid before filtering: 1000000
Number of pid after filtering: 10000


In [5]:
# Create train and test sets of playlists
test_size = 0.25
test_ind = int(len(df) * test_size)
curr_pid, prev_pid = df.loc[test_ind, 'pid'], df.loc[test_ind, 'pid']
while True:
    test_ind += 1
    curr_pid = df.loc[test_ind, 'pid']
    if curr_pid != prev_pid:
        break
    prev_pid = curr_pid
df.iloc[test_ind-1:test_ind+1, :]

,pid,track_uri
126077,323_329,spotify:track:5cZqsjVs6MevCnAkasbEOX
126078,323_381,spotify:track:0tmFHk3YWZcST0SZjViHhY


In [6]:
df_train = df.iloc[test_ind:, :].reset_index(drop = True)
df_test = df.iloc[:test_ind, :].reset_index(drop = True)
print("Test size in pid:", df_test['pid'].nunique() / len(pid))

Test size in pid: 0.2526


In [7]:
def df_to_dict(df, test_size = 10):
    train, test = {}, {}
    start_ind, end_ind = 0,0
    cur_pid, prev_pid = df.loc[0,'pid'], df.loc[0,'pid']
    for i in range(len(df)):
        if i % int(len(df) // 10) == 0: print(i)
        end_ind = i
        cur_pid = df.loc[i,'pid']
        if cur_pid != prev_pid:
            temp = df.loc[start_ind:end_ind-1, :]
            tracks = temp['track_uri'].values.tolist()
            train[prev_pid] = tracks[test_size:]
            test[prev_pid] = tracks[:test_size]
            start_ind = end_ind
        prev_pid = cur_pid
    return train, test

In [8]:
X_train, y_train = df_to_dict(df_train)
X_test, y_test = df_to_dict(df_test)

0
37820
75640
113460
151280
189100
226920
264740
302560
340380
378200
0
12607
25214
37821
50428
63035
75642
88249
100856
113463
126070


In [9]:
# Save to disk
train_test_data = (X_train, y_train, X_test, y_test)

with open(data_path + "train_test_data.pkl", "wb") as f:
    pickle.dump(train_test_data, f)

In [10]:
# Try loading
with open(data_path + "train_test_data.pkl", "rb") as f:
    X_train, y_train, X_test, y_test = pickle.load(f)